In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

In [27]:
df = pd.read_csv("Lottery Singapore Dataset.txt",sep = "\t", header=None)
df = np.flipud(df)
df = pd.DataFrame(df, columns = ['Game','Dates','Day', 'Numbers'])

print(df)
# df.head()
# columns = ("Name","Date","Day","Numbers")

            0           1          2     3
0   Singapore  11-08-2022   Thursday  8032
1   Singapore  10-08-2022  Wednesday  6844
2   Singapore  08-08-2022     Monday  2106
3   Singapore  07-08-2022     Sunday  9961
4   Singapore  06-08-2022   Saturday  3784
..        ...         ...        ...   ...
95  Singapore  31-03-2022   Thursday  4613
96  Singapore  30-03-2022  Wednesday  8730
97  Singapore  28-03-2022     Monday  9103
98  Singapore  27-03-2022     Sunday  1099
99  Singapore  26-03-2022   Saturday   660

[100 rows x 4 columns]


In [3]:
# df = df.drop(df.index[[0,]])
# df.columns = ("Name","Date","Day","Numbers")
print(type(df))
df.head()
# print(df.shape())
# df.types

<class 'pandas.core.frame.DataFrame'>


,Game,Dates,Day,Numbers
0,Singapore,26-03-2022,Saturday,660
1,Singapore,27-03-2022,Sunday,1099
2,Singapore,28-03-2022,Monday,9103
3,Singapore,30-03-2022,Wednesday,8730
4,Singapore,31-03-2022,Thursday,4613


In [4]:
df.tail()

,Game,Dates,Day,Numbers
95,Singapore,06-08-2022,Saturday,3784
96,Singapore,07-08-2022,Sunday,9961
97,Singapore,08-08-2022,Monday,2106
98,Singapore,10-08-2022,Wednesday,6844
99,Singapore,11-08-2022,Thursday,8032


In [5]:
print(df.shape)

(100, 4)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Game     100 non-null    object
 1   Dates    100 non-null    object
 2   Day      100 non-null    object
 3   Numbers  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [7]:
df.drop(["Game", "Dates", "Day"], axis=1, inplace=True)
df.head()

,Numbers
0,660
1,1099
2,9103
3,8730
4,4613


In [8]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
transformed_df.head()


,0
0,-1.453569
1,-1.309933
2,1.308899
3,1.186857
4,-0.160186


In [9]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

100

In [10]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

1

In [11]:
# Amount of games we need to take into consideration for prediction
window_length = 7
window_length

7

In [12]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [13]:
X.shape

(93, 7, 1)

In [14]:
y.shape


(93, 1)

In [15]:
X[0]


array([[-1.45356936],
       [-1.30993279],
       [ 1.30889887],
       [ 1.18685686],
       [-0.16018586],
       [-1.32890983],
       [ 1.31707863]])

In [16]:
y[0]


array([-0.64606355])

In [17]:
y[1]


array([-0.42062939])

In [18]:
# Init RNN
model = Sequential()
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [19]:
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [20]:
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [21]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = False)))

In [22]:
model.add(Dense(59))


In [23]:
model.add(Dense(number_of_features))


In [24]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy']) #0.0001

In [25]:
model.fit(x=X, y=y, batch_size=100, epochs=300, verbose=2) 

Epoch 1/300
1/1 - 54s - loss: 0.9625 - accuracy: 0.0000e+00
Epoch 2/300
1/1 - 1s - loss: 0.9617 - accuracy: 0.0000e+00
Epoch 3/300
1/1 - 0s - loss: 0.9606 - accuracy: 0.0000e+00
Epoch 4/300
1/1 - 0s - loss: 0.9601 - accuracy: 0.0000e+00
Epoch 5/300
1/1 - 0s - loss: 0.9585 - accuracy: 0.0000e+00
Epoch 6/300
1/1 - 0s - loss: 0.9576 - accuracy: 0.0000e+00
Epoch 7/300
1/1 - 1s - loss: 0.9568 - accuracy: 0.0000e+00
Epoch 8/300
1/1 - 0s - loss: 0.9560 - accuracy: 0.0000e+00
Epoch 9/300
1/1 - 1s - loss: 0.9547 - accuracy: 0.0000e+00
Epoch 10/300
1/1 - 1s - loss: 0.9536 - accuracy: 0.0000e+00
Epoch 11/300
1/1 - 1s - loss: 0.9534 - accuracy: 0.0000e+00
Epoch 12/300
1/1 - 0s - loss: 0.9522 - accuracy: 0.0000e+00
Epoch 13/300
1/1 - 1s - loss: 0.9519 - accuracy: 0.0000e+00
Epoch 14/300
1/1 - 1s - loss: 0.9510 - accuracy: 0.0000e+00
Epoch 15/300
1/1 - 0s - loss: 0.9489 - accuracy: 0.0000e+00
Epoch 16/300
1/1 - 0s - loss: 0.9480 - accuracy: 0.0000e+00
Epoch 17/300
1/1 - 0s - loss: 0.9473 - accuracy:

KeyboardInterrupt: 

In [ ]:
to_predict = df.tail(8)
to_predict
# y_pred = model.predict(np.array([scaled_to_predict]))
# print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])
# prediction = np.array(prediction)
# print("The actual numbers in the last lottery game were:", prediction[0])

In [ ]:
to_predict.drop([to_predict.index[0]],axis=0, inplace=True)
to_predict

In [ ]:
prediction = df.tail(1)
prediction

In [ ]:
to_predict = np.array(to_predict)
to_predict

In [ ]:
scaled_to_predict = scaler.transform(to_predict)
scaled_to_predict

In [ ]:
y_pred = model.predict(np.array([scaled_to_predict]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

In [ ]:
prediction = np.array(prediction)
print("The actual numbers in the last lottery game were:", prediction[0])

# 